In [1]:
import os
from pathlib import Path
import weaviate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import weaviate.classes as wvc
from weaviate.classes.config import Configure
import requests

In [11]:
def get_ollama_embedding(text):
    """Obtener embedding de Ollama directamente"""
    try:
        response = requests.post(
            "http://localhost:11434/api/embeddings",
            json={
                "model": "qwen3:4b",  # Usar el modelo que tienes disponible
                "prompt": text
            },
            timeout=30
        )
        if response.status_code == 200:
            return response.json()["embedding"]
        else:
            print(f"Error en Ollama: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error conectando a Ollama: {e}")
        return None
    
def load_new_file(file_path, collection_name):
    """
    Carga un nuevo documento PDF a una colección existente de Weaviate
    
    Args:
        file_path (str): Ruta al archivo PDF
        collection_name (str): Nombre de la colección existente
    """
    # Conectar a Weaviate
    client = weaviate.connect_to_local()
    print("✅ Conectado a Weaviate local")
    
    # Verificar que el archivo existe
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"No se encontró el archivo: {file_path}")
    
    # Verificar que la colección existe
    if not client.collections.exists(collection_name):
        raise ValueError(f"La colección '{collection_name}' no existe. Créala primero.")
    
    # Obtener la colección existente
    collection = client.collections.get(collection_name)
    print(f"📦 Conectado a la colección '{collection_name}'")
    
    # Cargar el PDF
    print(f"📄 Cargando PDF: {file_path}")
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    
    # Dividir el texto en chunks
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=350,
        chunk_overlap=50
    )
    chunked_documents = text_splitter.split_documents(docs)
    print(f"📝 Documento dividido en {len(chunked_documents)} chunks")
    
    # Obtener el nombre del archivo para el source
    file_name = Path(file_path).name
    
    # Obtener el siguiente chunk_id disponible (para no duplicar IDs)
    # Consultar todos los documentos para obtener el chunk_id más alto
    existing_docs = collection.query.fetch_objects(
        limit=10000  # Ajustar según tus necesidades
    )
    max_chunk_id = -1
    for obj in existing_docs.objects:
        if 'chunk_id' in obj.properties:
            max_chunk_id = max(max_chunk_id, obj.properties['chunk_id'])
    
    starting_chunk_id = max_chunk_id + 1
    print(f"🔢 Comenzando desde chunk_id: {starting_chunk_id}")
    
    # Insertar documentos
    print("💾 Insertando documentos...")
    with collection.batch.dynamic() as batch:
        for i, doc in enumerate(chunked_documents):
            batch.add_object(
                properties={
                    "text": doc.page_content,
                    "chunk_id": starting_chunk_id + i,
                    "source": file_name,
                    "length": len(doc.page_content)
                }
            )
    
    print(f"✅ {len(chunked_documents)} documentos insertados exitosamente")
    
    # Mostrar estadísticas de la colección
    total_objects = collection.aggregate.over_all(total_count=True)
    print(f"📊 Total de documentos en la colección: {total_objects.total_count}")
    
    return True

In [9]:
client = weaviate.connect_to_local()
collection_name = "DocumentosPDFOllama"

c:\Users\Jose\Documents\Programacion\Chatbot_Langchain\.venv\Lib\site-packages\weaviate\warnings.py:302: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


In [ ]:
client.collections.list_all()


{'DocumentosPDF': _CollectionConfigSimple(name='DocumentosPDF', description=None, generative_config=None, properties=[_Property(name='text', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer=None, vectorizer_configs={}), _Property(name='chunk_id', description=None, data_type=<DataType.INT: 'int'>, index_filterable=True, index_range_filters=False, index_searchable=False, nested_properties=None, tokenization=None, vectorizer_config=None, vectorizer=None, vectorizer_configs={}), _Property(name='source', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer=None, vectorizer_configs={}), _Property(name='length', description=None, data_type=<DataTy

In [18]:
query_embedding = get_ollama_embedding("What is a RAG?")

In [13]:
file_path = "./data/diego_velazquez.pdf"
load_new_file(file_path=file_path, collection_name=collection_name)

C:\Users\Jose\AppData\Local\Programs\Python\Python313\Lib\json\decoder.py:361: ResourceWarning: unclosed <socket.socket fd=580, family=23, type=1, proto=0, laddr=('::1', 51546, 0, 0), raddr=('::1', 8080, 0, 0)>
  obj, end = self.scan_once(s, idx)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)


✅ Conectado a Weaviate local
📦 Conectado a la colección 'DocumentosPDFOllama'
📄 Cargando PDF: ./data/diego_velazquez.pdf
📝 Documento dividido en 20 chunks
🔢 Comenzando desde chunk_id: 20
💾 Insertando documentos...
✅ 20 documentos insertados exitosamente
📊 Total de documentos en la colección: 40


True

c:\Users\Jose\Documents\Programacion\Chatbot_Langchain\.venv\Lib\site-packages\weaviate\warnings.py:302: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


In [ ]:
collection = client.collections.get(collection_name)
if query_embedding:
    response = collection.query.near_vector(
        near_vector=query_embedding,
        limit=4,
        return_metadata=wvc.query.MetadataQuery(distance=True)
    )
    
    print("\n📋 Resultados con embeddings de Ollama:")
    print(f"Objetos: {len(response.objects)}")
    print("=" * 50)
    for i, obj in enumerate(response.objects, 1):
        print(f"\n🔸 Resultado {i}:")
        print(f"   Distancia: {obj.metadata.distance:.4f}")
        print(f"   Texto: {obj.properties['text'][:200]}...")

In [ ]:
import os
from dotenv import load_dotenv

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Weaviate
from langchain.chains import RetrievalQA

# Carga las variables de entorno para las claves de API
load_dotenv()

# Configura tus claves de API
# Asegúrate de que OPENAI_API_KEY, WEAVIATE_URL y WEAVIATE_API_KEY estén en tu archivo .env

# --- 1. Cargar documentos ---
# Usaremos un archivo de texto simple como ejemplo
# Crea un archivo llamado 'ejemplo.txt' con el texto que quieras
# Por ejemplo: "LangChain es un framework para desarrollar aplicaciones con LLMs. Weaviate es un motor de búsqueda de vectores..."
loader = TextLoader("ejemplo.txt")
documents = loader.load()
print(f"Documentos cargados: {len(documents)}")

# --- 2. Dividir los documentos ---
# Esto es crucial para manejar textos largos.
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(f"Fragmentos creados: {len(docs)}")

# --- 3. Generar embeddings ---
# Inicializa el modelo de embeddings de OpenAI
embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)

# --- 4. Almacenar en Weaviate ---
# Asegúrate de que tu instancia de Weaviate esté funcionando.
# El nombre de la clase es el "esquema" para tus datos en Weaviate.
index_name = "LangChain_RAG_Demo"

# Crea la conexión con Weaviate y sube los documentos
# Esto creará una nueva colección (clase) en tu instancia de Weaviate
weaviate_client = Weaviate.from_documents(
    docs,
    embeddings_model,
    client=None, # Puedes pasar un cliente Weaviate ya inicializado aquí
    by_text=False, # Si los documentos ya tienen embeddings, usa True
    weaviate_url=weaviate_url,
    api_key=weaviate_api_key,
    index_name=index_name
)
weaviate_client.

print(f"Documentos almacenados en Weaviate bajo la clase '{index_name}'.")

# --- 5. Crear el sistema RAG ---
# Inicializa el modelo de lenguaje (LLM) que usará LangChain
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)

# Crea el "retriever" (recuperador). LangChain usará este objeto
# para buscar los documentos relevantes en Weaviate.
retriever = weaviate_client.as_retriever()

# Crea la cadena RAG. Le decimos que use el LLM para generar la respuesta
# y el retriever para encontrar la información relevante.
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

# --- Preguntar ---
query = "Explícame qué es LangChain."
response = qa_chain.invoke(query)

print("\n--- Respuesta del sistema RAG ---")
print(response['result'])